<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_04_YogFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

#Filtering out charts with specific yogs



For those who may not have access to a live MongoDB instance, we have set up a mechanism where the JSON file that would have been loaded into a hosted MongoDB instance, will be loaded into a locally (that is in the VM) installed MongoDB instance and the queries can be run in a manner that is identical to a live, hosted MongoDB instance

#SetUp Environment

##Install Software

In [1]:
!pip -qq install pyswisseph                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
!pip -qq install python-docx                #https://python-docx.readthedocs.io/en/latest/
!/usr/bin/python3 -m pip install "pymongo[srv]"
!apt install mongodb > /dev/null            # install local MongoDB server           
#!service mongodb start                      # start Local MongoDB server instance


     |████████████████████████████████| 1.8 MB 4.9 MB/s 
     |████████████████████████████████| 5.6 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 5.0 MB/s 




##Install, import prerequisites

In [2]:
#Utility functions 
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz

#import matplotlib.pyplot as plt
#import math


In [3]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py 
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py
# ---------------------------

!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png             
!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png       

In [26]:
import p21
import p21swe
import p21utils
import p21utilsR
import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21swe)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
importlib.reload(p21YogInfo)

p21swe imported
p21utils imported
p21utilsR imported
9 Yogs on record


<module 'p21YogInfo' from '/content/p21YogInfo.py'>

In [ ]:
#!cat p21YogInfo.py

#Local or Remote Atlas

##Local Database Operations

In [5]:
!service mongodb start                      # start Local MongoDB server instance

 * Starting database mongodb
   ...done.


In [6]:
mongoURI = "mongodb://localhost:27017/"                               # for remote server
client = MongoClient(mongoURI)

db = client.parashar21
#pd.DataFrame(list(db.list_collections()))

###Download Horoscope data in JSON and load into MongoDB

In [7]:
# Retrieve JSON file from Github and load in local Mongo
#!wget -O peopleData.json -q https://github.com/prithwis/parashar21/raw/main/data/peopleDataTest5.json

#!gdown -O peopleData.json https://drive.google.com/uc?id=1_VBHE7Hyue8t3hdBdLU6lgpnaWhhRVd4  # 39662 documents OLD file
!gdown  -O peopleData.json 1GIPjxHenAm2EKdtx8T0_RwkzkmCRGEH7  # 39662 documents
!mongoimport --db parashar21 --collection khona21 --drop --file peopleData.json

Downloading...
From: https://drive.google.com/uc?id=1GIPjxHenAm2EKdtx8T0_RwkzkmCRGEH7
To: /content/peopleData.json
100% 143M/143M [00:01<00:00, 135MB/s]
2022-09-21T12:06:15.229+0000	connected to: localhost
2022-09-21T12:06:15.229+0000	dropping: parashar21.khona21
2022-09-21T12:06:18.228+0000	[####....................] parashar21.khona21	28.3MB/137MB (20.7%)
2022-09-21T12:06:21.227+0000	[#########...............] parashar21.khona21	55.5MB/137MB (40.6%)
2022-09-21T12:06:24.231+0000	[##############..........] parashar21.khona21	83.2MB/137MB (60.9%)
2022-09-21T12:06:27.233+0000	[###################.....] parashar21.khona21	111MB/137MB (81.3%)
2022-09-21T12:06:29.891+0000	[########################] parashar21.khona21	137MB/137MB (100.0%)
2022-09-21T12:06:29.891+0000	imported 39662 documents


In [8]:
p21.kollection = db.khona21                   # Set the collection to work with
#kollection = db.khona21                   # Set the collection to work with
totalCharts = p21.kollection.count_documents({})
dbSize = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
print(totalCharts, ' documents,',round(dbSize,2), " KB data")

39662  documents, 146582.36  KB data


##Remote Atlas Database 
https://account.mongodb.com/account/login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# -------------------------------------------
# copy credentials file
!cp /content/drive/'My Drive'/Praxis/WebCredentials/Parashar21_Atlas_18Sep22.py credentials.py
# -------------------------------------------
# MongoDB credentials are stored here
#
from credentials import parashar21URI
# -------------------------------------------
mongoURI = parashar21URI                                   # for remote ATlas server

client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']
# ----------------------------------------------

Mounted at /content/drive


['Parashar21', 'admin', 'local']

In [ ]:
# Show existing Collections
db = client.Parashar21
#pd.DataFrame(list(db.list_collections()))
#kollection = db.khona21Test                   # Set the collection to work with
p21.kollection = db.khona21                   # Set the collection to work with
totalCharts = p21.kollection.count_documents({})
print(totalCharts)

39662


In [ ]:
#print(p21.kollection)

Collection(Database(MongoClient(host=['cluster0-shard-00-01.yarln.mongodb.net:27017', 'cluster0-shard-00-00.yarln.mongodb.net:27017', 'cluster0-shard-00-02.yarln.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-djz97v-shard-0', tls=True), 'Parashar21'), 'khona21')


#Report Generation Function
Please note that the generated report MS Office doc format **will be stored in the Colab VM drive** and needs to be downloaded to local machine for viewing

In [9]:
def genYogReport(yText, ChartStyle,Sample = True):
    
    if Sample:
        df4Report = df.sample(5)
    else:
        df4Report = df
    
    selectedCharts = len(df)
    printCharts = len(df4Report)
    #printStatus = str(printCharts)+' charts of total '+str(totalCharts)+' printed.'
    printStatus = str(len(df))+' charts of total '+str(totalCharts)+' selected and '+str(printCharts)+' printed'
    
    

    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21Yog_'+yText[0:4]+'_'+ChartStyle[0:1]+'_'+now.strftime("%H%M%S")+'.doc'
    curQueryStr = json.dumps(selCond)[0:100]
    # ---------------------------------------------------------
    p21utilsR.R01_CreateReportDoc(yText,printStatus)                      # Creates MS Word Doc called p21.document
    # ----------------------------------------------------------
    for chart in range(len(df4Report)):
        chartData =df4Report.iloc[chart]
        #R51_processChart(chartData)
        p21utilsR.R511_parseChartData(chartData)              # Break up database data into useful pieces
                                            # Creates two primary dicts, GLon, GRet
        for ctype in ['Rashi','Navamsa']:
        #for ctype in ['Rashi']:
            p21.ChartType = ctype
            p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                                # Rashi Num as well as Rashi name
            if ChartStyle == 'Bengal':
                p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
            if ChartStyle == 'South':
                p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
            if ChartStyle == 'North':
                p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
            #
            
                                                
        #print(p21.GrahaBhava)
        #print(p21.LordBhav)

        p21utilsR.R30_LocateYogs()
        p21utilsR.R512_FormatPage()
        #print('----')
    # ------------------------------------
    p21.document.save(fileName)
    


#Retrieve Charts 



In [27]:
%%capture
yogName = 'Adhi6'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
if (len(df) > 0):
    genYogReport(p21YogInfo.yogText[yogName],'Bengal')
else:
    print('no charts found')

##Chandra Mangal

In [10]:
%%capture
yogName = 'ChandraMangal'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'ChandraMangal2'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'ChandraMangal1'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

##Yogs based on the Moon

In [ ]:
%%capture
yogName = 'Kemadruma'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'Dhurdhura'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'Anapha'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'Sunapha'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

## Gajakesari Yog

In [ ]:
%%capture
#from p21YogInfo import  yogCond, yogText
yogName = 'Gajakesari'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
print(p21.GRashiN)

{'La': 9, 'Su': 11, 'Mo': 7, 'Ma': 4, 'Me': 1, 'Ju': 1, 'Ve': 3, 'Sa': 3, 'Ra': 4, 'Ke': 10}


## zBudhaditya Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'BudhAditya'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'BudhAdityaNR'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##zRaj Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'Rajyog'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##Chandra Mangal (Mahalakshmi) Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'ChandraMangal2'
#yogName = 'ChandraMangal1' # this returns no records in our database
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##Dhan Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'DhanYog'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##All Yogs

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
#yogList = ['BudhAditya','BudhAdityaNR', 'Gajakesari', 'Rajyog', 'ChandraMangal2','DhanYog']
for yogName in list(p21YogInfo.yogCond.keys()):
    #selCond = yogCond[yogName]
    df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
    #print(yogName, len(df))
    if len(df) > 0:
        genYogReport(yogText[yogName],'Bengal')


#Tests

In [ ]:
%%capture
yogText = 'Dhan^ yog : Lords 1 with 2 ^ ref : LHAE KN Rao, pg 77'
selCond = { 'BLConjunctsBL2.1': {'$in': [2,3]}        }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogText,'Bengal')

In [ ]:
%%capture
yogText = 'Dummy^ yog : Ma not conjuct with Moon ^ ref : https://www.astroyogi.com/kundli/yog/mahalaxmi'
selCond = {"$and":
                    [
                        {'GConjunctsG2.Ma' : {'$in': ['Su']}} ,     # Ma not conjunct Mo 
                        {'GConjunctsG2.Ma' : {'$nin': ['Mo']}} ,     # Ma not conjunct Mo        
                        {'GrahaBhava.Mo' : {'$nin': [1,2,3,4,5,6,7,8,9,10]}}     # Mo not in 3,6,8,12
                        
                    ]
         }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogText,'Bengal')

In [ ]:
%%capture
yogText = 'Dummy^ yog : Lords 1 with 2,3 ^ ref : LHAE KN Rao, pg 77'
selCond = { 'BLConjunctsBL2.1': {'$in': ['2','3']} }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogText,'Bengal')

In [ ]:
!rm *.doc